## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import image
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

In [2]:
import math

def grayscale(img):

    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    
def canny(img, low_threshold, high_threshold):
    
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices1, vertices2):

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color 
    cv2.fillPoly(mask, [vertices1,vertices2], ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=6):
   
    left_slope=[]
    right_slope=[]
    left_center=[]
    right_center=[]
    left_len=[]
    right_len=[]
    slope_eps=0.35
    def length(x1,y1,x2,y2):
        return math.sqrt((x2-x1)**2+(y2-y1)**2)
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=(y2-y1)/(x2-x1)
            center=(x1+x2)/2,(y1+y2)/2 
            if slope<-slope_eps:
                left_slope.append(slope)
                left_center.append(center)
                left_len.append(length(x1,y1,x2,y2))
            elif slope>slope_eps:
                right_slope.append(slope)
                right_center.append(center)
                right_len.append(length(x1,y1,x2,y2))
    left_slope=np.array(left_slope)
    right_slope=np.array(right_slope)
    start_y=img.shape[0]*0.65
    end_y=img.shape[0]
    if len(left_slope>0):
        left_cnt=0
        left_center_mean=0,0
        left_slope_mean=0
        for i in range(len(left_slope)):
            #left_center_mean=left_center_mean[0]+left_len[i]*left_center[i][0],left_center_mean[1]+left_len[i]*left_center[i][1]#is it useful?
            #left_slope_mean+=left_slope[i]*left_len[i]#is it useful?
            left_cnt+=left_len[i]
        if left_cnt>0:
            left_slope_mean=np.mean(left_slope)
            left_center_mean=np.mean([c[0] for c in left_center]),np.mean([c[1] for c in left_center])
            left_start=int((start_y-left_center_mean[1])/left_slope_mean+left_center_mean[0]) , int(start_y)
            left_end=int((end_y-left_center_mean[1])/left_slope_mean+left_center_mean[0]) , int(end_y)
            cv2.line(img,left_start,left_end, color, thickness)
           
        
    if len(right_slope>0):
        right_center_mean=0,0
        right_slope_mean=0
        right_cnt=0
        for i in range(len(right_slope)):
            #right_center_mean=right_center_mean[0]+right_len[i]*right_center[i][0],right_center_mean[1]+right_len[i]*right_center[i][1]#is it useful?
            #right_slope_mean+=right_slope[i]*right_len[i]#is it useful?
            right_cnt+=right_len[i]
        if right_cnt>0:
            right_slope_mean=np.mean(right_slope)
            right_center_mean=np.mean([c[0] for c in right_center]),np.mean([c[1] for c in right_center])
            right_start=int((start_y-right_center_mean[1])/right_slope_mean+right_center_mean[0]) , int(start_y)
            right_end=int((end_y-right_center_mean[1])/right_slope_mean+right_center_mean[0]) , int(end_y)
            cv2.line(img, right_start, right_end, color, thickness)
           
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
   
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

In [3]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

## Build a Lane Finding Pipeline



In [15]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
import matplotlib.image as mimg
def Lane_finding(img,kernel_size=7,low_threshold=50,high_threshold=150,rho=2,
                 theta=np.pi/180,threshold=15,min_line_len=60,max_line_gap=30):
    imshape=img.shape
    gray_img=grayscale(img)
    blur_img=gaussian_blur(gray_img,kernel_size)
    edges=canny(blur_img, low_threshold, high_threshold)
    vertics1=np.array([[(0,imshape[0]),(imshape[1]/2-20, imshape[0]*0.65), (imshape[1]/2+20, imshape[0]*0.65), (imshape[1], imshape[0])]], dtype=np.int32)
    vertics2=np.array([[(imshape[1]/2-140,imshape[0]),(imshape[1]/2+25, imshape[0]*0.72), (imshape[1]/2+25, imshape[0]*0.72), (imshape[1]/2+240, imshape[0])]], dtype=np.int32)
    edges=region_of_interest(edges,vertics1,vertics2)
    line_img=hough_lines(edges,rho,theta,threshold,min_line_len,max_line_gap)
    return line_img


In [16]:
input_path='test_images/'
output_path='test_images_output'
test_images=os.listdir(input_path)
for img in test_images:
    path=input_path+'\\'+img
    image=mpimg.imread(path)
    output_img=Lane_finding(image)
    output=output_path+'\\'+img
    mimg.imsave(output,output_img)

## Test Videos

In [17]:
# Import everything needed to edit/save/watch video clips

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [18]:
def process_image(image):
   
    line_img=Lane_finding(image)
    result=weighted_img(line_img,image)
    return result

Let's try the one with the solid white lane on the right first ...

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [19]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4




  0%|                                                                                          | 0/251 [00:00<?, ?it/s]

  1%|▋                                                                                 | 2/251 [00:00<00:18, 13.15it/s]

  2%|█▎                                                                                | 4/251 [00:00<00:16, 14.97it/s]

  2%|█▉                                                                                | 6/251 [00:00<00:15, 15.86it/s]

  3%|██▌                                                                               | 8/251 [00:00<00:15, 16.02it/s]

  4%|███▏                                                                             | 10/251 [00:00<00:15, 16.06it/s]

  5%|███▊                                                                             | 12/251 [00:00<00:14, 16.45it/s]

  6%|████▌                                                                            | 14/251 [00:00<00:14, 16.46it/s]

  6%|█████▏                   

 81%|█████████████████████████████████████████████████████████████████               | 204/251 [00:24<00:05,  8.20it/s]

 82%|█████████████████████████████████████████████████████████████████▎              | 205/251 [00:25<00:05,  8.19it/s]

 82%|█████████████████████████████████████████████████████████████████▋              | 206/251 [00:25<00:05,  8.18it/s]

 83%|██████████████████████████████████████████████████████████████████▎             | 208/251 [00:25<00:05,  8.22it/s]

 83%|██████████████████████████████████████████████████████████████████▌             | 209/251 [00:25<00:05,  8.22it/s]

 84%|███████████████████████████████████████████████████████████████████▎            | 211/251 [00:25<00:04,  8.21it/s]

 84%|███████████████████████████████████████████████████████████████████▌            | 212/251 [00:26<00:04,  8.15it/s]

 85%|███████████████████████████████████████████████████████████████████▉            | 213/251 [00:26<00:04,  8.16it/s]

 86%|███████████████████████████

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 34.2 s


In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))